# Kerasによるニューラルネット実装例(MNIST)

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

In [10]:
# 自分で設定するパラメータ
batch_size = 128
num_classes = 10
epochs = 5

In [11]:
# データ抽出
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# one-hot-label に変換
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [13]:
# モデルの構築
model = Sequential()

# 最初の隠れ層(入力784 -> 出力512)
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))

# 隠れ層(入力512 -> 出力512)
## 隠れ層は追加可能
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

# 出力層(入力512 -> 出力10)
model.add(Dense(10, activation='softmax'))

# モデル概要表示
model.summary()

# モデルのコンパイル
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

# フィッティング(学習)
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

# スコアの計算・表示
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 10s - loss: 0.2477 - acc: 0.9

In [7]:
# 学習モデルの保存
model.save("trained_data")

# モデルのテスト

In [9]:
# 画像の読み込み
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

#Jupyterでインライン表示するための宣言
%matplotlib inline 

#画像の読み込み
im = Image.open("./tegaki33.jpg")

In [7]:
gray_img = im.convert('L')
gray_img = gray_img.resize((28,28))
#画像をarrayに変換
im_list = np.array(gray_img, 'f')

### 入力画像によって変更

In [ ]:
##白地に黒文字の場合は実行
##白黒反転をし，正規化
im_list = 1 - im_list/256

##黒地に白文字の場合は実行
##正規化
im_list = im_list/256

In [ ]:
#入力画像の表示
plt.imshow(im_list,cmap=plt.get_cmap('gray'))

In [6]:
# 画像判別
test = im_list.reshape(1,784)
print(model.predict(test))
print(np.argmax(model.predict(test)[0]))

[[  1.58248413e-15   4.12705223e-17   1.00000000e+00   1.96933303e-10
    4.36465308e-20   1.13435567e-14   6.66654065e-10   4.95727811e-15
    2.81834642e-18   1.75679753e-21]]
2
